In [5]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [57]:
import numpy as np
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Conv2D, BatchNormalization, MaxPooling2D, Conv2D
from keras.layers import Lambda
from keras.engine.topology import InputLayer
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras import backend as K
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.vgg16 import VGG16, preprocess_input as vgg16_preprocess_input, decode_predictions as vgg16_decode_predictions
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils import np_utils

## ============== Net From Scratch For MNIST ==============

### Data Preparation

In [1]:
from keras.datasets import mnist

In [75]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [76]:
## add one color channel - we are in grayscale
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [77]:
## one hot encode output
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [78]:
## normalize input ??
px_mean = x_train.mean().astype('float32')
px_std = x_train.std().astype('float32')

In [56]:
def pixel_normalize(x): return (x-px_mean) / px_std

### Linear Model

In [88]:
layers = [
    InputLayer(input_shape=(28,28,1)),
    Lambda(pixel_normalize),
    Flatten(),
    Dense(10, activation='softmax')
]

model = Sequential(layers)
model.summary()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
lambda_7 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                7850      
Total params: 7,850
Trainable params: 7,850
Non-trainable params: 0
_________________________________________________________________


In [80]:
gen = ImageDataGenerator()

In [81]:
train_gen = gen.flow(x_train, y_train, batch_size=64)
test_gen = gen.flow(x_test, y_test, batch_size=64)

In [89]:
model.fit_generator(train_gen, train_gen.n // 64, epochs=1, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/1
937/937 [==============================] - 4s 4ms/step - loss: 0.4305 - acc: 0.8714 - val_loss: 0.3003 - val_acc: 0.9143


In [91]:
model.optimizer.lr = 0.1
model.fit_generator(train_gen, train_gen.n // 64, epochs=4, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/4
937/937 [==============================] - 4s 4ms/step - loss: 0.2994 - acc: 0.9151 - val_loss: 0.2912 - val_acc: 0.9179
Epoch 2/4
937/937 [==============================] - 4s 4ms/step - loss: 0.2848 - acc: 0.9199 - val_loss: 0.2812 - val_acc: 0.9198
Epoch 3/4
937/937 [==============================] - 4s 4ms/step - loss: 0.2788 - acc: 0.9213 - val_loss: 0.2746 - val_acc: 0.9249
Epoch 4/4
937/937 [==============================] - 4s 4ms/step - loss: 0.2725 - acc: 0.9246 - val_loss: 0.2730 - val_acc: 0.9250


### Single Dense Layer

In [94]:
layers = [
    InputLayer(input_shape=(28,28,1)),
    Lambda(pixel_normalize),
    Flatten(),
    Dense(28*28, activation='relu'),
    Dense(10, activation='softmax')
]
model = Sequential(layers)
model.summary()
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
lambda_9 (Lambda)            (None, 28, 28, 1)         0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 784)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 784)               615440    
_________________________________________________________________
dense_27 (Dense)             (None, 10)                7850      
Total params: 623,290
Trainable params: 623,290
Non-trainable params: 0
_________________________________________________________________


In [95]:
model.fit_generator(train_gen, train_gen.n // 64, epochs=1, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/1
937/937 [==============================] - 5s 5ms/step - loss: 0.1911 - acc: 0.9419 - val_loss: 0.1345 - val_acc: 0.9589


In [96]:
model.optimizer.lr = 0.1
model.fit_generator(train_gen, train_gen.n // 64, epochs=4, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0789 - acc: 0.9747 - val_loss: 0.0913 - val_acc: 0.9723
Epoch 2/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0555 - acc: 0.9824 - val_loss: 0.0859 - val_acc: 0.9743
Epoch 3/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0412 - acc: 0.9867 - val_loss: 0.1008 - val_acc: 0.9692
Epoch 4/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0351 - acc: 0.9883 - val_loss: 0.0921 - val_acc: 0.9734


In [97]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, train_gen.n // 64, epochs=4, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0299 - acc: 0.9900 - val_loss: 0.0795 - val_acc: 0.9796
Epoch 2/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0250 - acc: 0.9917 - val_loss: 0.0841 - val_acc: 0.9781
Epoch 3/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0216 - acc: 0.9929 - val_loss: 0.0862 - val_acc: 0.9789
Epoch 4/4
937/937 [==============================] - 5s 5ms/step - loss: 0.0228 - acc: 0.9927 - val_loss: 0.0921 - val_acc: 0.9807


### Convolutions

In [108]:
Dropout?

In [127]:
def get_convolution_model(dropout=0.3):
    layers = [
        InputLayer(input_shape=(28,28,1)),
        Lambda(pixel_normalize),
        Conv2D(32,3,activation='relu'),
        BatchNormalization(),
        Conv2D(32,3,activation='relu'),
        MaxPooling2D(),
        BatchNormalization(),
        Conv2D(64,3,activation='relu'),
        BatchNormalization(),
        Conv2D(64,3,activation='relu'),
        MaxPooling2D(),
        Flatten(),
        BatchNormalization(),
        Dropout(rate=dropout-0.2),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(rate=dropout),
        Dense(10, activation='softmax')
    ]
    model = Sequential(layers)
    model.summary()
    return model

model = get_convolution_model(dropout=0.5)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 28, 28, 1)         0         
_________________________________________________________________
lambda_18 (Lambda)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_35 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
batch_normalization_36 (Batc (None, 12, 12, 32)        128       
__________

In [128]:
model.fit_generator(train_gen, train_gen.n // 64, epochs=1, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/1
937/937 [==============================] - 26s 28ms/step - loss: 0.1346 - acc: 0.9602 - val_loss: 0.0319 - val_acc: 0.9891


In [129]:
model.optimizer.lr = 0.1
model.fit_generator(train_gen, train_gen.n // 64, epochs=4, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/4
937/937 [==============================] - 25s 26ms/step - loss: 0.0514 - acc: 0.9841 - val_loss: 0.0315 - val_acc: 0.9890
Epoch 2/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0451 - acc: 0.9865 - val_loss: 0.0246 - val_acc: 0.9917
Epoch 3/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0384 - acc: 0.9878 - val_loss: 0.0408 - val_acc: 0.9872
Epoch 4/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0342 - acc: 0.9896 - val_loss: 0.0235 - val_acc: 0.9928


In [130]:
model.optimizer.lr = 0.01
model.fit_generator(train_gen, train_gen.n // 64, epochs=4, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/4
937/937 [==============================] - 24s 26ms/step - loss: 0.0308 - acc: 0.9906 - val_loss: 0.0191 - val_acc: 0.9945
Epoch 2/4
937/937 [==============================] - 21s 23ms/step - loss: 0.0273 - acc: 0.9914 - val_loss: 0.0225 - val_acc: 0.9931
Epoch 3/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0265 - acc: 0.9914 - val_loss: 0.0262 - val_acc: 0.9921
Epoch 4/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0247 - acc: 0.9922 - val_loss: 0.0177 - val_acc: 0.9947


In [131]:
model.optimizer.lr = 0.001
model.fit_generator(train_gen, train_gen.n // 64, epochs=4, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/4
937/937 [==============================] - 24s 26ms/step - loss: 0.0207 - acc: 0.9931 - val_loss: 0.0200 - val_acc: 0.9932
Epoch 2/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0205 - acc: 0.9935 - val_loss: 0.0233 - val_acc: 0.9923
Epoch 3/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0194 - acc: 0.9937 - val_loss: 0.0223 - val_acc: 0.9937
Epoch 4/4
937/937 [==============================] - 21s 22ms/step - loss: 0.0183 - acc: 0.9944 - val_loss: 0.0207 - val_acc: 0.9932


In [132]:
model.optimizer.lr = 0.0005
model.fit_generator(train_gen, train_gen.n // 64, epochs=2, 
                   validation_data=test_gen, validation_steps=test_gen.n // 64, workers = 4)

Epoch 1/2
937/937 [==============================] - 24s 25ms/step - loss: 0.0139 - acc: 0.9955 - val_loss: 0.0290 - val_acc: 0.9926
Epoch 2/2
937/937 [==============================] - 21s 22ms/step - loss: 0.0162 - acc: 0.9948 - val_loss: 0.0186 - val_acc: 0.9947


In [133]:
### Teraz bym zrobil ensamble